## Generate interactive bokeh plot
#### Last updated 2022-08-07
This notebook takes the output of the `ana_polar_LCD_stickerspacer_vM.ipynb` notebook and illustrates how one can generate interactive stand-alone html pages to provide a dynamic interface to play with the data.

Note that this notebook requires `bokeh` to use. To install run

    pip install bokeh 



In [ ]:
import pandas as pd 
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns

import bokeh
from bokeh.layouts import gridplot, layout
from bokeh.models import BoxSelectTool, LassoSelectTool, LinearColorMapper, TapTool, OpenURL, CustomJS, ColorBar
from bokeh.models import Column, ColumnDataSource, BasicTicker, Slider,  CustomJSFilter, CDSView, BooleanFilter
from bokeh.plotting import curdoc, figure, show,  output_file, save

In [ ]:
# open bokeh plot in notebook
bokeh.io.output_notebook()

### Setup variables

In [ ]:
#### PLOTTING SETTINGS ####
#  
# Holehouse Lab Specific Formating 
#
#
# Set such that PDF fonts export in a manner that they
# are editable in illustrator/affinity
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# set to define axes linewidths
matplotlib.rcParams['axes.linewidth'] = 0.5

# this defines some prefactors so inline figures look nice
# on a retina macbook. These can be commented out without any
# issue and are solely asthetic.
%matplotlib inline
%config InlineBackend.figure_format='retina'

# UPDATE 2020-12-31 (my preferred font is Avenir...)
font = {'family' : 'avenir',
    	'weight' : 'normal'}

matplotlib.rc('font', **font)

### COLOR PALETTES ###

aro_palette = sns.light_palette('#f2a139', reverse=False, as_cmap=False, n_colors=10)
aro_bokeh = sns.color_palette(aro_palette,10).as_hex()
charge_palette = sns.light_palette('#C03D3E', reverse=False, as_cmap=False, n_colors=10)
charge_bokeh = sns.color_palette(charge_palette,10).as_hex()
ali_palette = 'Greys'

## Generate interactive plot
The code below provides an example for generating an interactive plot where slides let you highlight specific residue types.

In [ ]:
# old data - kept for posterity although typo fix in
# header means this will fail if called...
#data = pd.read_csv('bokeh_data_vF.csv')
data = pd.read_csv('bokeh_data_2.csv')

# define color map 
exp_cmap = LinearColorMapper(palette=aro_bokeh, low = min(data["fraction_aromatic"]), high = max(data["fraction_aromatic"]))


# read
data["protein"]           = data.protein
data["position"]          = data.position
data["fraction_aromatic"] = data.fraction_aromatic
data["relative_aromatic"]  = data.relative_aromatic
data["select_fraction_aromatic"] = ''
source = ColumnDataSource(data)


# define plotting text settings for hovertool
tooltips = """
<div>
    <span style="font-size: 15px;">@protein</span>&nbsp;
    <span style="font-size: 10px; color: #666;">(@position)</span>
</div>
<div>
    <span style="font-size: 11px;">Frac. aromatic:</span>&nbsp;
    <span style="font-size: 11px; font-weight: bold;">@fraction_aromatic{0.0000}</span>
<div>
    <div style="font-size: 11px;">Frac. aromatic / (fract. charge + f. aliphatic):</span>
    <span style="font-size: 11px;">@relative_aromatic{0.00}</span>
</div>
    <span style="font-size: 11px; color: #666;">@{info}
<div>
    <span style=>@{pp_seq}
"""

##
# define plot 
plot = figure(width=1200, height=600,
              toolbar_location='above', outline_line_color=None,
              tooltips=tooltips,
              y_axis_label="Frac. charge + frac. aliphatic residues",
              x_axis_label="Frac. aromatic residues")

plot.legend.location = "top_right"
plot.title.text = "Polar-rich low complexity domains with sliders for aromatic enrichment"
plot.title.text_font_size = "16px"


# add slider 
rel_threshold_slider = Slider(start=0.1, end=1, value=0.5, step=.01,default_size=10 ,title="Frac. aromatic / (frac. charge + frac. aliphatic)")

slider_step=1
FCR_aliphatic_percentile_slider = Slider(start=1, end=100, value=1, step=slider_step, default_size=50,title="Percentile for charge + aliphatic cutoff (i.e. percentile along Y-axis)")
p_slider_convertion = {p*slider_step:np.percentile(np.array(data['FCR_aliphatic']), p*slider_step) for p in range(int(100*slider_step))}

## define filter for data
custom_filter = CustomJSFilter(args=dict(slider=rel_threshold_slider, FCR_aliphatic_slider=FCR_aliphatic_percentile_slider,p_slider_convertion=p_slider_convertion), code="""
    const indices = []
    for (var i = 0; i < source.get_length(); i++) {
        if (source.data['FCR_aliphatic'][i] < p_slider_convertion[FCR_aliphatic_slider.value]) {
            if (source.data['relative_aromatic'][i] > slider.value) {
                indices.push(true)
            } else {
                indices.push(false)
            }
            
        } else {
          indices.push(false)
        } 
    }
    return indices
""")

#define calling of plotted data 
view = CDSView(source=source, filters=[custom_filter])

# force a re-render when the sliders changes
rel_threshold_slider.js_on_change('value', CustomJS(args=dict(source=source), code="""
   source.change.emit()
   """))

# FCR_aliphatic_percentile_slider.on_change('value',scale_slider)
FCR_aliphatic_percentile_slider.js_on_change('value', CustomJS(args=dict(source=source), code="""
   source.change.emit()
   """))

##
# plot data
allLCD = plot.circle(y="FCR_aliphatic", x="fraction_aromatic", size=7, source=source, level="overlay", line_color=None,
                    fill_color={'field'  : "select_fraction_aromatic", 'transform' : exp_cmap})

# plot selection
LCD = plot.circle(y="FCR_aliphatic", x="fraction_aromatic", size=7, source=source, level="overlay", line_color=None, view=view,
                    fill_color={'field'  : "fraction_aromatic", 'transform' : exp_cmap})

# add selected annotated LCD labels # text = SelectedName
LCDtext = plot.text(y="FCR_aliphatic", x="fraction_aromatic", x_offset=10, y_offset=-5,
          text="protein", text_align="left", text_baseline="middle", view=view,
          text_font_size="12px", source=source)


# add hover 
plot.hover.renderers = [allLCD]

# if point is clicked send to uniprot link 
open_url = CustomJS(args=dict(source=source), code="""
    source.inspected.indices.forEach(function(index) {
    const protein = source.data["protein"][index];
    const url = "https://www.uniprot.org/uniprot/" + encodeURIComponent(protein);
    window.open(url);
    });
    """)


# add tools functions
plot.add_tools(TapTool(callback=open_url, renderers=[LCD], behavior="inspect"))

# add colorbar
color_bar = ColorBar(color_mapper=exp_cmap, ticker=BasicTicker(desired_num_ticks=10),title="Aromatic Fraction")
plot.add_layout(color_bar, 'right')


layout = Column(rel_threshold_slider, FCR_aliphatic_percentile_slider, plot)
curdoc().add_root(layout)
show(layout)
## saving the plot 
output_file(filename="html_out/relative_aromatic_bokehplot_vf.html", title="Aromatic_HTML_file")
save(layout)